In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

In [2]:
dir_data_stock = "data/kaggle0/Stocks"
dir_data_etfs = "data/kaggle0/ETFs"
aapl = "/aapl.us.txt"

In [3]:
data_aapl = pd.read_csv(dir_data_stock + aapl) 
data_aapl['Date'] = [datetime.strptime(d, '%Y-%m-%d') for d in data_aapl['Date']]

data_aapl.tail()

,Date,Open,High,Low,Close,Volume,OpenInt
8359,2017-11-06,171.75,174.36,171.10,173.63,34901241,0
8360,2017-11-07,173.29,174.51,173.29,174.18,24424877,0
8361,2017-11-08,174.03,175.61,173.71,175.61,24451166,0
8362,2017-11-09,174.48,175.46,172.52,175.25,29533086,0
8363,2017-11-10,175.11,175.38,174.27,174.67,25130494,0


In [7]:
class Trail:
    period = 5
    hist = []
    
    def __init__(self, period=None, init=None):
        if init is not None:
            self.hist = init
        
        if period is not None:
            self.period = period
    
    def push(self, val):
        self.hist.append(val)

        while len(self.hist) > self.period:
            self.hist = self.hist[1:]
    
    def decision(self):
        if len(self.hist) < self.period:
            return None
        
        if self.hist[0] > self.hist[-1]:
            return 'buy'
        else:
            return 'sell'


In [12]:
class Sim:
    cash = 100000.0
    holding = 0
    date = datetime.strptime('2000-01-01', '%Y-%m-%d')
    end = datetime.strptime('2017-01-01', '%Y-%m-%d')
    trail = Trail()
    bankrupt_date = None
    
    def __init__(self):
        pass
    
    def step(self):
        self.date += timedelta(days=1)
        
        price = data_aapl[data_aapl['Date']==self.date]#['Close']
        print(price)
        self.trail.push(price)
        
        directive = self.trail.decision()
        
        if directive == 'buy':
            if price < self.cash:
                # can buy
                self.cash -= price
                self.holding += 1
            else:
                # no cash
                if self.holding == 0:
                    self.bankrupt_date = self.date
            
        elif directive == 'sell':
            self.cash += price
            self.holding -= 1

    def report(self):
        print(self.cash)
        
    def run(self):
        while self.date <= self.end:
            self.step()

        self.report()


In [13]:
sim = Sim()
sim.run()

Empty DataFrame
Columns: [Date, Open, High, Low, Close, Volume, OpenInt]
Index: []


/var/folders/5f/7b9dcjmd1c78j41h7tb02gycnrxd5n/T/ipykernel_8404/692015211.py:22: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  if self.hist[0] > self.hist[-1]:


TypeError: Invalid comparison between dtype=datetime64[ns] and ndarray